In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
!pip install keras-tuner
import keras_tuner as kt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/widsTeam_15/finalized_df_wids/"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Merged_ros_df.csv  Merged_rus_df.csv  Merged_test_df.csv  Merged_train_df_unbalanced.csv


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/widsTeam_15/finalized_df_wids/Merged_rus_df.csv")
test_df = pd.read_csv("/content/drive/MyDrive/widsTeam_15/finalized_df_wids/Merged_test_df.csv")

In [ ]:
#checking for NA
print("NA for training df: ", train_df.isna().sum().sum())
print("NA for testing df: ", test_df.isna().sum().sum())

NA for training df:  0
NA for testing df:  0


In [ ]:
print(train_df.isna().sum())

participant_id                      0
PreInt_Demos_Fam_Child_Ethnicity    0
Basic_Demos_Enroll_Year_2016        0
Basic_Demos_Enroll_Year_2017        0
Basic_Demos_Enroll_Year_2018        0
                                   ..
197throw_198thcolumn                0
197throw_199thcolumn                0
198throw_199thcolumn                0
ADHD_Outcome                        0
Sex_F                               0
Length: 19975, dtype: int64


In [ ]:
print(train_df.shape)
print(test_df.shape)

(574, 19975)
(304, 19973)


In [ ]:
test_df.columns

Index(['participant_id', 'PreInt_Demos_Fam_Child_Ethnicity',
       'Basic_Demos_Enroll_Year_2016', 'Basic_Demos_Enroll_Year_2017',
       'Basic_Demos_Enroll_Year_2018', 'Basic_Demos_Enroll_Year_2019',
       'Basic_Demos_Enroll_Year_2020', 'Basic_Demos_Study_Site_2',
       'Basic_Demos_Study_Site_3', 'Basic_Demos_Study_Site_4',
       ...
       '195throw_196thcolumn', '195throw_197thcolumn', '195throw_198thcolumn',
       '195throw_199thcolumn', '196throw_197thcolumn', '196throw_198thcolumn',
       '196throw_199thcolumn', '197throw_198thcolumn', '197throw_199thcolumn',
       '198throw_199thcolumn'],
      dtype='object', length=19973)

In [ ]:
train_df.columns

Index(['participant_id', 'PreInt_Demos_Fam_Child_Ethnicity',
       'Basic_Demos_Enroll_Year_2016', 'Basic_Demos_Enroll_Year_2017',
       'Basic_Demos_Enroll_Year_2018', 'Basic_Demos_Enroll_Year_2019',
       'Basic_Demos_Enroll_Year_2020', 'Basic_Demos_Study_Site_2',
       'Basic_Demos_Study_Site_3', 'Basic_Demos_Study_Site_4',
       ...
       '195throw_198thcolumn', '195throw_199thcolumn', '196throw_197thcolumn',
       '196throw_198thcolumn', '196throw_199thcolumn', '197throw_198thcolumn',
       '197throw_199thcolumn', '198throw_199thcolumn', 'ADHD_Outcome',
       'Sex_F'],
      dtype='object', length=19975)

In [ ]:
X_train = train_df.drop(columns=['participant_id', 'ADHD_Outcome', 'Sex_F'])
Y_train = train_df[['ADHD_Outcome','Sex_F']]
X_test = test_df.drop(columns=['participant_id'])
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(574, 19972)
(574, 2)
(304, 19972)


In [ ]:
#standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Sequential()

#input layer
input_layer = keras.layers.InputLayer(input_shape=(19972,), name='input')
model.add(input_layer)

#hidden layers
hidden_layer_1 = keras.layers.Dense(units=128, activation='relu',name='hidden_layer_1')
model.add(hidden_layer_1)

hidden_layer_2 = keras.layers.Dense(units= 64, activation='relu', name='hidden_layer_2')
model.add(hidden_layer_2)

hidden_layer_3 = keras.layers.Dense(units=32, activation='relu', name = 'hidden_layer_3')
model.add(hidden_layer_3)

hidden_layer_4 = keras.layers.Dense(units=16, activation='relu', name = 'hidden_layer_4')
model.add(hidden_layer_4)

#output layer for binary classification
output_layer = keras.layers.Dense(units=2, activation='sigmoid', name='output')
model.add(output_layer)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)               │ (None, 128)                 │       2,556,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_2 (Dense)               │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_3 (Dense)               │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_4 (Dense)               │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,567,442 (9.79 MB)

 Trainable params: 2,567,442 (9.79 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 50
model.compile(optimizer=Adam(learning_rate = .005), loss='binary_crossentropy', metrics=['binary_accuracy'])

#training the model
model_hist = model.fit(X_train, Y_train, epochs= num_epochs, batch_size=40, validation_split=0.2, verbose=1)

predictions = model.predict(X_test)
train_predictions = model.predict(X_train)

predicted_ADHD = (predictions[:,1] > 0.5).astype(int)
predicted_Sex_F = (predictions[:,0] > 0.5).astype(int)

train_predicted_ADHD = (train_predictions[:,1] > 0.5).astype(int)
train_predicted_Sex_F = (train_predictions[:,0] > 0.5).astype(int)





Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - binary_accuracy: 0.4982 - loss: 0.7220 - val_binary_accuracy: 0.3217 - val_loss: 0.8424
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - binary_accuracy: 0.7573 - loss: 0.6460 - val_binary_accuracy: 0.4826 - val_loss: 0.6918
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - binary_accuracy: 0.8405 - loss: 0.3609 - val_binary_accuracy: 0.5391 - val_loss: 0.6891
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - binary_accuracy: 0.9007 - loss: 0.2343 - val_binary_accuracy: 0.5435 - val_loss: 0.7018
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - binary_accuracy: 0.9482 - loss: 0.1163 - val_binary_accuracy: 0.5435 - val_loss: 0.6920
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - binary_accuracy: 0.9664 - loss: 0.0754 - val_binary_accuracy: 0.5391 - val_loss: 0.6965
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - binary_accuracy: 0.9645 - loss: 0.0689 - val_binary_accuracy: 0.5348 - val_loss: 0.7031
Epoch 8/50

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

actual_ADHD = Y_train['ADHD_Outcome']
actual_Sex_F = Y_train['Sex_F']

adhd_f1 = f1_score(actual_ADHD, train_predicted_ADHD)
sex_f_f1 = f1_score(actual_Sex_F, train_predicted_Sex_F)

print("ADHD F1 Score: ", adhd_f1)
print("Sex F1 Score: ", sex_f_f1)

# Calculate metrics
accuracy_sex = accuracy_score(actual_Sex_F, train_predicted_Sex_F)
precision_sex = precision_score(actual_Sex_F, train_predicted_Sex_F)
recall_sex = recall_score(actual_Sex_F, train_predicted_Sex_F)

print(f"Accuracy for sex: {accuracy_sex:.2f}")
print(f"Precision for sex: {precision_sex:.2f}")
print(f"Recall for sex: {recall_sex:.2f}")


accuracy_adhd = accuracy_score(actual_ADHD, train_predicted_ADHD)
precision_adhd = precision_score(actual_ADHD, train_predicted_ADHD)
recall_adhd = recall_score(actual_ADHD, train_predicted_ADHD)
print()

print(f"Accuracy for ADHD: {accuracy_adhd:.2f}")
print(f"Precision for ADHD: {precision_adhd:.2f}")
print(f"Recall for ADHD: {recall_adhd:.2f}")




ADHD F1 Score:  0.436036036036036
Sex F1 Score:  0.5133858267716536
Accuracy for sex: 0.46
Precision for sex: 0.47
Recall for sex: 0.57

Accuracy for ADHD: 0.45
Precision for ADHD: 0.44
Recall for ADHD: 0.44


In [ ]:
#creating a dataframe with participant id and predictions

test_predictions = test_df[['participant_id']].copy()
test_predictions['participant_id'] = test_df['participant_id']
test_predictions['ADHD_Outcome'] = predicted_ADHD
test_predictions['Sex_F'] = predicted_Sex_F

test_predictions.to_csv("submission.csv", index=False)
print("Prediction saved to submission.csv")

Prediction saved to submission.csv


In [ ]:
test_predictions

,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,0,1
1,vhGrzmvA3Hjq,0,1
2,ULliyEXjy4OV,1,0
3,LZfeAb1xMtql,1,1
4,EnFOUv0YK1RG,0,0
...,...,...,...
299,UadZfjdEg7eG,1,1
300,IUEHiLmQAqCi,0,1
301,cRySmCadYFRO,0,1
302,E3MvDUtJadc5,0,1
